In [118]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from torch_geometric.datasets import Planetoid

In [100]:
dataset =Planetoid(".","Cora",split='random',num_val=500,num_test=0)
data=dataset[0]
num_class=7

In [280]:
# am=np.eye(2708)
am=np.zeros((2708,2708)).astype(np.float32)
for i in range(data.edge_index.shape[1]):
    am[data.edge_index[0,i],data.edge_index[1,i]]=1

x=np.array([np.array(data.x)])
y=np.zeros((1,2708,7)).astype(np.float32)
for i in range(2708):
    y[0,i,data.y[i]]=1
trains=np.array(data.train_mask)
vals=np.array(data.val_mask)
train_inds=[[0,i] for i in range(2708) if vals[i]==False]
val_inds=[[0,i] for i in range(2708) if vals[i]==True]

In [356]:
class gnn_layer(tf.keras.layers.Layer): 
   def __init__(self,**kwargs): 
      super(gnn_layer, self).__init__(**kwargs) 
      
   def call(self, inps):
      q, k=inps
      out=K.batch_dot(am,k,axes=[1,1])+q
      return out


inp=tf.keras.layers.Input((2708, 1433))
q=tf.keras.layers.Conv1D(64,kernel_size=1,activation="relu")(inp)
k=tf.keras.layers.Conv1D(64,kernel_size=1,activation="relu")(inp)
gnn=gnn_layer()([q,k])

q=tf.keras.layers.Conv1D(64,kernel_size=1,activation="relu")(gnn)
k=tf.keras.layers.Conv1D(64,kernel_size=1,activation="relu")(gnn)
gnn=gnn_layer()([q,k])

out=tf.keras.layers.Dense(7,activation="softmax")(gnn)
model=tf.keras.Model(inp,out)
@tf.function
def loss(y_pred,y_true):
    # return tf.losses.CategoricalCrossentropy()(y_pred[:,np.logical_not(vals),:],y_true[:,np.logical_not(vals),:])
    return tf.losses.CategoricalCrossentropy()(tf.gather_nd(params=y_pred,indices=train_inds),tf.gather_nd(params=y_true,indices=train_inds))
@tf.function
def metric(y_pred,y_true):
    # return tf.keras.metrics.CategoricalAccuracy()(y_pred[:,vals,:],y_true[:,vals,:])
    return K.mean(tf.cast(K.argmax(tf.gather_nd(params=y_pred,indices=val_inds),axis=-1)==K.argmax(tf.gather_nd(params=y_true,indices=val_inds),axis=-1),tf.float32))
    # return tf.losses.CategoricalCrossentropy()(tf.gather_nd(params=y_pred,indices=val_inds),tf.gather_nd(params=y_true,indices=val_inds))


model.compile(
        optimizer=tf.keras.optimizers.Adam(0.01),
        loss=loss,
        metrics=[metric],
    )
model.summary()

Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           [(None, 2708, 1433)] 0                                            
__________________________________________________________________________________________________
conv1d_82 (Conv1D)              (None, 2708, 64)     91776       input_43[0][0]                   
__________________________________________________________________________________________________
conv1d_83 (Conv1D)              (None, 2708, 64)     91776       input_43[0][0]                   
__________________________________________________________________________________________________
gnn_layer_40 (gnn_layer)        (None, 2708, 64)     0           conv1d_82[0][0]                  
                                                                 conv1d_83[0][0]           

In [357]:
model.fit(x,y,epochs=64)

Epoch 1/32
1/1 [==============================] - 10s 10s/step - loss: 4.5155 - metric: 0.1160
Epoch 2/32
1/1 [==============================] - 4s 4s/step - loss: 4.0985 - metric: 0.3260
Epoch 3/32
1/1 [==============================] - 4s 4s/step - loss: 5.7743 - metric: 0.3220
Epoch 4/32
1/1 [==============================] - 4s 4s/step - loss: 4.8793 - metric: 0.4380
Epoch 5/32
1/1 [==============================] - 4s 4s/step - loss: 4.3931 - metric: 0.3740
Epoch 6/32
1/1 [==============================] - 4s 4s/step - loss: 3.7136 - metric: 0.4360
Epoch 7/32
1/1 [==============================] - 4s 4s/step - loss: 3.1676 - metric: 0.5080
Epoch 8/32
1/1 [==============================] - 4s 4s/step - loss: 2.5528 - metric: 0.5700
Epoch 9/32
1/1 [==============================] - 4s 4s/step - loss: 1.6223 - metric: 0.6700
Epoch 10/32
1/1 [==============================] - 4s 4s/step - loss: 0.9318 - metric: 0.7780
Epoch 11/32
1/1 [==============================] - 4s 4s/step - los